[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/I2VGen-XL-colab/blob/main/I2VGen_XL_Vid2Vid_colab.ipynb)

In [ ]:
%cd /content

!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/damo-video-to-video/resolve/main/open_clip_pytorch_model.bin -d /root/.cache/modelscope/hub/damo/Video-to-Video -o open_clip_pytorch_model.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/damo-video-to-video/resolve/main/non_ema_0035000.pth -d /root/.cache/modelscope/hub/damo/Video-to-Video -o non_ema_0035000.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/damo-video-to-video/resolve/main/v2-1_512-ema-pruned.ckpt -d /root/.cache/modelscope/hub/damo/Video-to-Video -o v2-1_512-ema-pruned.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/damo-video-to-video/raw/main/configuration.json -d /root/.cache/modelscope/hub/damo/Video-to-Video -o configuration.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/damo-video-to-video/resolve/main/.mdl -d /root/.cache/modelscope/hub/damo/Video-to-Video -o .mdl
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/damo-video-to-video/resolve/main/.msc -d /root/.cache/modelscope/hub/damo/Video-to-Video -o .msc
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/damo-video-to-video/resolve/main/assets/images/Fig_1.png -d /root/.cache/modelscope/hub/damo/Video-to-Video -o assets/images/Fig_1.png

!pip install -q xformers==0.0.20 triton==2.0.0 gradio torchsde open_clip_torch einops rotary-embedding-torch fairscale 
!pip install -q pytorch-lightning torchmetrics -U
!pip install https://github.com/camenduru/I2VGen-XL-colab/releases/download/v1.0/modelscope-1.9.1-py3-none-any.whl

# https://huggingface.co/spaces/damo-vilab/MS-Vid2Vid-XL-demo/blob/main/app.py modified
import tempfile
import cv2
import gradio as gr
from modelscope.outputs import OutputKeys
from modelscope.pipelines import pipeline

pipe = pipeline(task="video-to-video", model='damo/Video-to-Video', model_revision='v1.1.0', device='cuda:0')

def check_input_video(video_path: str) -> None:
    cap = cv2.VideoCapture(video_path)
    n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    cap.release()
    if n_frames != 32 or width != 448 or height != 256:
        raise gr.Error(
            f"Input video must be 32 frames of size 448x256. Your video is {n_frames} frames of size {width}x{height}."
        )

def video_to_video(video_path: str, text: str) -> str:
    check_input_video(video_path)
    p_input = {"video_path": video_path, "text": text}
    output_file = tempfile.NamedTemporaryFile(suffix=".mp4", delete=False)
    pipe(p_input, output_video=output_file.name)[OutputKeys.OUTPUT_VIDEO]
    return output_file.name

with gr.Blocks(css="style.css") as demo:
    with gr.Group():
        input_video = gr.Video(label="Input video", type="filepath")
        text_description = gr.Textbox(label="Text description")
        run_button = gr.Button()
        output_video = gr.Video(label="Output video")
    text_description.submit(
        fn=check_input_video,
        inputs=input_video,
        queue=False,
        api_name=None,
    ).success(
        fn=video_to_video,
        inputs=[input_video, text_description],
        outputs=output_video,
        api_name=None,
    )
    run_button.click(
        fn=check_input_video,
        inputs=input_video,
        queue=False,
        api_name=None,
    ).success(
        fn=video_to_video,
        inputs=[input_video, text_description],
        outputs=output_video,
        api_name="run",
    )

if __name__ == "__main__":
    demo.queue(max_size=10).launch(share=True)